In [55]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import time
import pandas as pd
import numpy as np
import re

## 색상 팔레트 가져오기 (웹크롤링)

In [60]:
#브라우저 꺼짐 방지 
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

#페이지 열기
link="https://colorhunt.co/palettes/popular"

driver= wd.Chrome(options=chrome_options)
driver.get(link)

In [61]:
#Popular - All time
driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/div[1]/div[3]').click()

In [62]:
#무한 스크롤
import time

SCROLL_PAUSE_TIME = 2

# Get scroll height (마지막 시점의 창 높이 저장)
last_height = driver.execute_script("return document.body.scrollHeight")         

i = 68
while True:

    # Scroll down to bottom (창 높이까지 스크롤)                                                    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page (스크롤 후 창 로딩될 때까지 기다림)
    time.sleep(SCROLL_PAUSE_TIME)                                                
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")  
    # time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height (스크롤이 된 후에 창 높이를 새로운 높이로 저장)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if int(driver.find_element(By.XPATH, f'/html/body/div[4]/div[2]/div/div[{i}]/div[2]/div/div/span').text.replace(',','')) < 10000: #하트 수가 1만개 이상만 가져오기
        break
    elif new_height == last_height: #새로운 높이가 이전 높이와 변하지 않았으면 스크롤 종료
        break
    i+=40
    
    last_height = new_height

In [63]:
#rgb 색상 추출 함수화
def color_extract(data):
    rgb_code = re.search(r'rgb\(((\d+,\s*\d+,\s*\d+))\)', data).group(1)
    rgb_code = rgb_code.split(",")
    new_rgb = [int(x) for x in rgb_code]
    return new_rgb

In [64]:
#팔레트 추출
palette=[]

for i in range(32,370): #총 338개의 팔레트
    palette_table={}

    try:
        #팔레트 색상 추출
        c0 = driver.find_element(By.CSS_SELECTOR, f'body > div.wrap.main.flex > div.page > div > div:nth-child({i}) > div.palette > div.place.c0').get_attribute('style')
        c1 = driver.find_element(By.CSS_SELECTOR, f'body > div.wrap.main.flex > div.page > div > div:nth-child({i}) > div.palette > div.place.c1').get_attribute('style')
        c2 = driver.find_element(By.CSS_SELECTOR, f'body > div.wrap.main.flex > div.page > div > div:nth-child({i}) > div.palette > div.place.c2').get_attribute('style')
        c3 = driver.find_element(By.CSS_SELECTOR, f'body > div.wrap.main.flex > div.page > div > div:nth-child({i}) > div.palette > div.place.c3').get_attribute('style')

        palette_table['c0'] = color_extract(c0)
        palette_table['c1'] = color_extract(c1)
        palette_table['c2'] = color_extract(c2)
        palette_table['c3'] = color_extract(c3)
    
        palette.append(palette_table)
    
    except Exception as e:
        print("에러발생")
        continue

In [67]:
palette_df = pd.DataFrame.from_dict(palette)
palette_df.to_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/웹크롤링_색상팔레트.csv")